# Demonstration of ResNet with 2 Inputs

### Imports

In [1]:
import numpy as np
import os
from DataGenerator_Outp2 import DataGenerator

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, train_test_split
from tensorflow import  squeeze
from tensorflow.keras.layers import Input, Conv1D, Reshape, LayerNormalization, ReLU, BatchNormalization, Add, AveragePooling1D, Flatten, Dense, GRU, concatenate, Dropout 
from keras.regularizers import l2
from keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
import tensorflow as tf

### Loading subset

In [2]:
path_main = "C:/Users/vogel/Desktop/Study/Master BMIT/1.Semester/Programmierprojekt/feat_new/"
files = os.listdir(path_main+"derivations/dev0")

### Functions for building model

In [3]:
# ResNet Block
def resnet_block(inp, batch_size, num_filter, kernel_sizes=[8,5,3], pool_size=3, pool_stride_size=2):
    def conv_block(inp, batch_size, num_filter, kernel_size):
        outp = Conv1D(kernel_size=kernel_size, filters=num_filter, padding="same")(inp)
        outp = BatchNormalization()(outp)
        outp = ReLU()(outp)       
        return outp
    
    def create_output_long(inp, batch_size, num_filter, kernel_sizes):
        outp = inp
        for i in range(0, len(kernel_sizes)):
            outp = conv_block(outp, batch_size, num_filter, kernel_sizes[i])
        return outp
    
    def create_output_short(inp, num_filter):
        outp = Conv1D(kernel_size=1, filters=num_filter, padding="same")(inp)
        outp = BatchNormalization()(outp)
        return outp
    
    outp_long = create_output_long(inp, batch_size, num_filter, kernel_sizes)
    outp_short = create_output_short(inp, num_filter)

    outp = Add()([outp_long, outp_short])

    if len(outp.shape) == 4:     
        outp = squeeze(outp, axis=1)

    outp = AveragePooling1D(pool_size=pool_size, strides=pool_stride_size)(outp)
    
    return outp

# Spectral Block
def spec_temp_block(inp, batch_size):

    outp = tf.signal.stft(inp, frame_length=128, frame_step=64)
    outp = tf.abs(outp)
    outp = LayerNormalization()(outp)
    outp = Reshape((8*batch_size,65))(outp)
    outp = GRU(64)(outp)
    outp = Flatten()(outp)
    outp = Dense(32, activation="relu", kernel_regularizer=l2(0.001))(outp)
    outp = BatchNormalization()(outp)
    
    return outp
   
# Create blocks of the network 
def net_blocks(batch_size):
    inp_time = Input(shape=(batch_size, 624), name="input_time0")
    inp_spec = Input(shape=(batch_size, 624), name="input_spec0")

    
    outp_time = resnet_block(inp_time, batch_size=batch_size, num_filter=64)
    outp_spec = spec_temp_block(inp_spec, batch_size=batch_size)

    for i in range(0,4):
        outp_time = resnet_block(outp_time, batch_size=batch_size, num_filter=128)

    model_time = keras.Model(inp_time, outp_time)
    model_spec = keras.Model(inp_spec, outp_spec)
        
    return model_time, model_spec, inp_time, inp_spec, outp_time, outp_spec

# Create Batches
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

### Build and train model

In [4]:
# Initialize kfold
n_splits = 3
kfold = KFold(n_splits=n_splits)
kfold.get_n_splits(files)

# Initialize Hyperparameter
kernel_init = "he_uniform"
batch_size = 256
l2_lambda = 0.001   


all_mae_sbp, all_mae_dbp = [], []
for train_index, test_index in kfold.split(files): 
    
    # Separate training, validation and test ids
    train_index, val_index = train_test_split(train_index, test_size=0.2)
    train_id = [files[x] for x in train_index]
    val_id = [files[x] for x in val_index]
    test_id = [files[x] for x in test_index]
    
    # Initialize Datagenerator
    print("Loading Datagenerator")
    generator_train = DataGenerator(path_main, train_id, batch_size=batch_size)
    generator_val = DataGenerator(path_main, val_id, batch_size=batch_size)
    generator_test = DataGenerator(path_main, test_id, batch_size=batch_size)

    # Build ResNet blocks
    print("Creating Model")
    model_time, model_spec, input_time, input_spec, outp_time, outp_spec = net_blocks(batch_size)
    
    # Concatenate blocks and build rest of the model
    outp_time = Reshape((1, outp_time.shape[1]*outp_time.shape[2]))(outp_time)
    outp_time = squeeze(outp_time, axis=1)
    merged = concatenate([outp_time, outp_spec])
    merged_outp = Dense(32, activation="relu", kernel_regularizer=l2(l2_lambda), kernel_initializer=kernel_init)(merged)
    merged_outp = Dropout(0.2)(merged_outp)
    merged_outp = Dense(32, activation="relu", kernel_regularizer=l2(l2_lambda), kernel_initializer=kernel_init)(merged_outp)
    merged_outp = Dropout(0.2)(merged_outp)
    merged_outp = Dense(2, activation='relu')(merged_outp)
    
    final_model = keras.Model(inputs=[input_time,
                            input_spec],
                            outputs=merged_outp,
                            name='Final_Model')
    
    # Initialize training
    optimizer = optimizers.RMSprop(learning_rate=0.0001)
    es = EarlyStopping(monitor="mae", patience=1)
    final_model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    final_model.fit(generator_train, 
                    validation_data=generator_val,
                    epochs=5,
                    verbose=1, 
                    callbacks=[es])
    
    all_pred = []
    all_true = []
    
    # Prediction
    for batch_index in range(generator_test.__len__()):
        #print(batch_index)
        batch_data, batch_true_labels = generator_test.__getitem__(batch_index)

        batch_pred = final_model.predict(batch_data, verbose=0)
        all_pred.append(batch_pred)
        sbp_mean = np.mean(batch_true_labels[:,0])
        dbp_mean = np.mean(batch_true_labels[:,1])
        all_true.append(np.array([sbp_mean, dbp_mean]))

    all_pred = np.concatenate(all_pred, axis=0)
    all_true = np.array(all_true)

    mae_sbp = mean_absolute_error(all_pred[:,0], all_true[:,0])
    mae_dbp = mean_absolute_error(all_pred[:,1], all_true[:,1])

    all_mae_sbp.append(mae_sbp)
    all_mae_dbp.append(mae_dbp)

mean_mae_sbp = np.mean(all_mae_sbp)
mean_mae_dbp = np.mean(all_mae_dbp)

Loading Datagenerator
Creating Model
Counting Subjects
Counting Subjects
Epoch 1/5
423/423 [==============================] - 180s 410ms/step - loss: 7344.9292 - mae: 78.5229 - val_loss: 18587.7773 - val_mae: 97.7537
Epoch 2/5
423/423 [==============================] - 150s 354ms/step - loss: 2080.6411 - mae: 35.7300 - val_loss: 1176759.2500 - val_mae: 907.4972
Epoch 3/5
423/423 [==============================] - 152s 360ms/step - loss: 1538.9128 - mae: 34.1828 - val_loss: 54911.7070 - val_mae: 183.7365
Epoch 4/5
423/423 [==============================] - 149s 353ms/step - loss: 910.4427 - mae: 23.1567 - val_loss: 1398532.7500 - val_mae: 926.6061
Epoch 5/5
423/423 [==============================] - 149s 352ms/step - loss: 1519.7834 - mae: 34.3120 - val_loss: 13251.0645 - val_mae: 95.8319
Counting Subjects
Loading Datagenerator
Creating Model
Counting Subjects
Counting Subjects
Epoch 1/5
778/778 [==============================] - 280s 352ms/step - loss: 3598.8455 - mae: 55.2480 - val_lo

### Results

In [5]:
print("Mean MAE of SBP: ", mean_mae_sbp)
print("Mean MAE of DBP: ", mean_mae_dbp)

Mean MAE of SBP:  55.74029124720925
Mean MAE of DBP:  49.85096200643385
